# Анализ пользовательского поведения в e-commerce

**Цель проекта:** Исследовать поведение пользователей в интернет-магазине и выявить факторы, влияющие на принятия решения о покупке.

**Данные:** Датасет содержит события взаимодействия пользователей с товарами.
Каждая строка соответствует отдельному событию:
просмотру товара, добавлению в корзину или покупке. Данные загружаются из SQL-запроса в pandas DataFrame для дальнейшего анализа.

**Инструменты:** Python (pandas), SQL (SQLite)

## 1. Загрузка и первичный осмотр данных

Перед выполнением расчетов был проведен базовый анализ данных
для оценки структуры датасета и качества данных

In [ ]:
# Импортируем необходимые библиотеки
import pandas as pd
import sqlite3

# Загружаем данные из CSV файла
events_df = pd.read_csv('events.csv')

# Выводим первые 5 строк для ознакомления со структурой
events_df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [ ]:
# Получаем информацию о датасете: типы данных, количество записей, наличие пропусков
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299999 entries, 0 to 299998
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   event_time     299999 non-null  object 
 1   event_type     299999 non-null  object 
 2   product_id     299999 non-null  int64  
 3   category_id    299999 non-null  int64  
 4   category_code  204488 non-null  object 
 5   brand          257826 non-null  object 
 6   price          299999 non-null  float64
 7   user_id        299999 non-null  int64  
 8   user_session   299999 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 20.6+ MB


В датасете присутствуют пропущенные значения в полях: категория товара и бренд.
Данные пропуски не влияют на расчет ключевых метрик, поэтому были оставлены без обработки.

Критических проблем с качеством данных, препятствующих дальнейшему анализу, не выявлено.

### Ограничение выборки
Для ускорения анализа ограничиваем данные первыми 200,000 записей

In [ ]:
# Ограничиваем датасет до 200 тысяч записей для оптимизации производительности
events_df = events_df.head(200_000)

# Проверяем размерность данных
events_df.shape

(200000, 9)

In [ ]:
# Анализируем соотношение просмотров, добавлений в корзину и покупок
events_df['event_type'].value_counts()

,count
event_type,
view,193377
purchase,3639
cart,2984


In [ ]:
# Преобразуем строковое представление времени в формат datetime для дальнейшего анализа
events_df['event_time'] = pd.to_datetime(events_df['event_time'])
events_df['event_time'].head()

,event_time
0,2019-10-01 00:00:00+00:00
1,2019-10-01 00:00:00+00:00
2,2019-10-01 00:00:01+00:00
3,2019-10-01 00:00:01+00:00
4,2019-10-01 00:00:04+00:00


In [ ]:
# Создаем отдельный столбец с датой (без времени) для группировки по дням
events_df['event_date'] = events_df['event_time'].dt.date
events_df[['event_time', 'event_date']].head()

,event_time,event_date
0,2019-10-01 00:00:00+00:00,2019-10-01
1,2019-10-01 00:00:00+00:00,2019-10-01
2,2019-10-01 00:00:01+00:00,2019-10-01
3,2019-10-01 00:00:01+00:00,2019-10-01
4,2019-10-01 00:00:04+00:00,2019-10-01


In [ ]:
# Создаем подключение к базе данных SQLite
db_connection = sqlite3.connect('ecommerce.db')

# Загружаем датафрейм в таблицу 'events'
events_df.to_sql('events', db_connection, if_exists='replace', index=False)

200000

## 4. SQL-анализ: Основные метрики
### 4.1 Распределение типов событий

In [ ]:
# Подсчитываем количество каждого типа событий с помощью SQL
query_event_types = """
    SELECT
        event_type,
        COUNT(*) AS events_count
    FROM events
    GROUP BY event_type
    ORDER BY events_count DESC
"""

pd.read_sql(query_event_types, db_connection)

,event_type,events_count
0,view,193377
1,purchase,3639
2,cart,2984


### 4.2 Топ категорий по покупкам

Цель анализа: определить, где сосредоточен основной пользовательский интерес,
перед переходом к более глубокому поведенческому анализу

In [ ]:
# Анализируем, какие категории товаров чаще всего покупают, выводим первые 10 категорий
query_top_categories = """
    SELECT
        category_code,
        COUNT(*) AS purchases_count
    FROM events
    WHERE event_type = 'purchase'
      AND category_code IS NOT NULL
    GROUP BY category_code
    ORDER BY purchases_count DESC
    LIMIT 10
"""

pd.read_sql(query_top_categories, db_connection).head(10)

,category_code,purchases_count
0,electronics.smartphone,1665
1,electronics.audio.headphone,151
2,computers.notebook,97
3,appliances.kitchen.washer,87
4,electronics.clocks,79
5,electronics.video.tv,76
6,appliances.environment.vacuum,57
7,appliances.kitchen.refrigerators,51
8,appliances.environment.water_heater,22
9,appliances.environment.air_heater,21


### 4.3 Топ брендов по категориям

In [ ]:
# Находим самые популярные бренды в каждой категории
query_top_brands_by_category = """
    SELECT
        category_code, brand,
        COUNT(*) AS purchases_count
    FROM events
    WHERE event_type = 'purchase'
      AND category_code IS NOT NULL
    GROUP BY category_code, brand
    ORDER BY purchases_count DESC
    LIMIT 10
"""

pd.read_sql(query_top_brands_by_category, db_connection).head(10)

,category_code,brand,purchases_count
0,electronics.smartphone,samsung,728
1,electronics.smartphone,apple,540
2,electronics.smartphone,xiaomi,200
3,electronics.smartphone,huawei,113
4,electronics.audio.headphone,apple,81
5,electronics.smartphone,oppo,44
6,computers.notebook,acer,40
7,electronics.audio.headphone,xiaomi,37
8,electronics.clocks,apple,28
9,electronics.video.tv,samsung,27


### 4.4 Конверсия из корзины в покупку по категориям
Рассчитываем, какой процент товаров из корзины действительно покупается. Это классическая e-commerce воронка: просмотр товара → добавление в корзину → покупка, которая делается для оценки пользовательского пути

In [ ]:
# Анализируем конверсию: сколько товаров из добавленных в корзину было куплено
query_cart_to_purchase_conversion = """
    SELECT
        category_code,
        cart_count,
        purchase_count,
        ROUND(purchase_count * 1.0 / cart_count, 3) AS conversion_rate
    FROM (
        SELECT
            category_code,
            COUNT(*) FILTER (WHERE event_type = 'cart') AS cart_count,
            COUNT(*) FILTER (WHERE event_type = 'purchase') AS purchase_count
        FROM events
        GROUP BY category_code
    ) category_stats
    WHERE cart_count >= 10  -- Фильтруем категории с малым количеством событий
    ORDER BY conversion_rate DESC
    LIMIT 20
"""

pd.read_sql(query_cart_to_purchase_conversion, db_connection).head(20)

,category_code,cart_count,purchase_count,conversion_rate
0,computers.notebook,10,97,9.700
1,appliances.kitchen.washer,25,87,3.480
2,None,389,836,2.149
3,appliances.environment.water_heater,14,22,1.571
4,appliances.environment.vacuum,40,57,1.425
5,electronics.telephone,15,20,1.333
6,electronics.video.tv,72,76,1.056
7,electronics.clocks,78,79,1.013
8,electronics.tablet,20,18,0.900
9,electronics.audio.headphone,183,151,0.825


Анализ показал, что количество покупок превышает количество добавлений в корзину. Это означает, что добавление товара в корзину не является обязательным шагом перед покупкой.

Причинами могут быть прямая покупка товара или особенности трекинга событий. В связи с этим классическая интерпретация воронки неприменима для данного датасета

## 5. Анализ влияния цены на поведение
### 5.1 Диапазон цен

In [ ]:
# Определяем минимальную и максимальную цену в данных
query_price_range = """
    SELECT
        MAX(price) AS max_price,
        MIN(price) AS min_price
    FROM events
"""

pd.read_sql(query_price_range, db_connection)

,max_price,min_price
0,2574.07,0.0


### 5.2 Поведение пользователей по ценовым сегментам
Для более детального анализа поведения пользователей товары были сгруппированы
по ценовым сегментам. Это позволяет сравнивать паттерны покупок для товаров разной стоимости

In [ ]:
# Разбиваем товары на ценовые сегменты и анализируем поведение
query_behavior_by_price_segment = """
    SELECT
        price_segment,
        COUNT(*) FILTER (WHERE event_type = 'cart') AS cart_count,
        COUNT(*) FILTER (WHERE event_type = 'purchase') AS purchase_count
    FROM (
        SELECT
            event_type,
            CASE
                WHEN price < 100 THEN 'дешево'
                WHEN price >= 100 AND price < 500 THEN 'средне'
                WHEN price >= 500 AND price < 1000 THEN 'дорого'
                ELSE 'очень дорого'
            END AS price_segment
        FROM events
    ) events_with_segments
    GROUP BY price_segment
    ORDER BY cart_count DESC
"""

pd.read_sql(query_behavior_by_price_segment, db_connection)

,price_segment,cart_count,purchase_count
0,средне,1883,1956
1,дешево,538,1034
2,дорого,378,456
3,очень дорого,185,193


### 5.3 Воронка конверсии по категориям

In [ ]:
# Анализируем полную воронку конверсии через уникальные сессии
query_full_funnel_by_category = """
    SELECT
        category_code,
        COUNT(DISTINCT user_session) FILTER (WHERE event_type = 'view') AS unique_views,
        COUNT(DISTINCT user_session) FILTER (WHERE event_type = 'cart') AS unique_carts,
        COUNT(DISTINCT user_session) FILTER (WHERE event_type = 'purchase') AS unique_purchases
    FROM events
    WHERE category_code IS NOT NULL
    GROUP BY category_code
    ORDER BY unique_views DESC, unique_carts DESC, unique_purchases DESC
    LIMIT 20
"""

pd.read_sql(query_full_funnel_by_category, db_connection).head(20)

,category_code,unique_views,unique_carts,unique_purchases
0,electronics.smartphone,15486,1260,1411
1,electronics.clocks,1915,45,75
2,electronics.audio.headphone,1870,118,142
3,computers.notebook,1318,8,86
4,electronics.video.tv,1092,48,70
5,appliances.kitchen.refrigerators,950,1,43
6,appliances.environment.vacuum,843,25,57
7,apparel.shoes,824,0,13
8,appliances.kitchen.washer,799,12,71
9,auto.accessories.player,649,0,16


## 6. Анализ времени до покупки
Поскольку классическая воронка не полностью описывает поведение пользователей,
в анализ была введена альтернативная метрика - время до покупки.

Время до покупки определяется как разница между первым просмотром товара
и первой покупкой этого же товара одним и тем же пользователем.
Данная метрика отражает, сколько времени пользователю требуется
для принятия решения о покупке после первого контакта с товаром.

### 6.1 Время от первого просмотра до покупки

In [ ]:
# Рассчитываем время между первым просмотром товара и его покупкой
query_time_to_purchase = """
    SELECT
        *,
        ROUND((JULIANDAY(first_purchase_time) - JULIANDAY(first_view_time)) * 1440, 2) AS time_to_purchase_minutes
    FROM (
        SELECT
            user_id, product_id, category_code, brand, price,
            MIN(event_time) FILTER (WHERE event_type = 'view') AS first_view_time,
            MIN(event_time) FILTER (WHERE event_type = 'purchase') AS first_purchase_time
        FROM events
        GROUP BY user_id, product_id
    ) user_product_timeline
    WHERE first_purchase_time IS NOT NULL
      AND first_view_time IS NOT NULL
    ORDER BY time_to_purchase_minutes DESC
"""

time_to_purchase_data = pd.read_sql(query_time_to_purchase, db_connection)
time_to_purchase_data.head()

,user_id,product_id,category_code,brand,price,first_view_time,first_purchase_time,time_to_purchase_minutes
0,553079594,15300165,None,transcend,6.12,2019-10-01 00:07:02+00:00,2019-10-01 05:44:39+00:00,337.62
1,547477164,19200010,construction.tools.saw,huter,84.66,2019-10-01 02:32:57+00:00,2019-10-01 05:57:07+00:00,204.17
2,546370010,15800018,None,karcher,540.02,2019-10-01 02:23:31+00:00,2019-10-01 05:45:27+00:00,201.93
3,551349791,1004903,electronics.smartphone,huawei,111.79,2019-10-01 02:38:51+00:00,2019-10-01 05:50:49+00:00,191.97
4,514492564,12600087,appliances.kitchen.grill,redmond,113.23,2019-10-01 02:21:39+00:00,2019-10-01 05:28:39+00:00,187.00


### 6.2 Время до покупки с учетом ценовых сегментов

In [ ]:
# Расширяем анализ: добавляем ценовые сегменты к времени покупки
query_time_to_purchase_with_segments = """
    SELECT
        *,
        ROUND((JULIANDAY(first_purchase_time) - JULIANDAY(first_view_time)) * 1440, 2) AS time_to_purchase_minutes,
        CASE
            WHEN price < 100 THEN 'дешево'
            WHEN price >= 100 AND price < 500 THEN 'средне'
            WHEN price >= 500 AND price < 1000 THEN 'дорого'
            ELSE 'очень дорого'
        END AS price_segment
    FROM (
        SELECT
            user_id, product_id, category_code, brand, price,
            MIN(event_time) FILTER (WHERE event_type = 'view') AS first_view_time,
            MIN(event_time) FILTER (WHERE event_type = 'purchase') AS first_purchase_time
        FROM events
        GROUP BY user_id, product_id
    ) user_product_timeline
    WHERE first_purchase_time IS NOT NULL
      AND first_view_time IS NOT NULL
    ORDER BY time_to_purchase_minutes DESC
"""

purchase_journey_df = pd.read_sql(query_time_to_purchase_with_segments, db_connection)
purchase_journey_df.head()

,user_id,product_id,category_code,brand,price,first_view_time,first_purchase_time,time_to_purchase_minutes,price_segment
0,553079594,15300165,None,transcend,6.12,2019-10-01 00:07:02+00:00,2019-10-01 05:44:39+00:00,337.62,дешево
1,547477164,19200010,construction.tools.saw,huter,84.66,2019-10-01 02:32:57+00:00,2019-10-01 05:57:07+00:00,204.17,дешево
2,546370010,15800018,None,karcher,540.02,2019-10-01 02:23:31+00:00,2019-10-01 05:45:27+00:00,201.93,дорого
3,551349791,1004903,electronics.smartphone,huawei,111.79,2019-10-01 02:38:51+00:00,2019-10-01 05:50:49+00:00,191.97,средне
4,514492564,12600087,appliances.kitchen.grill,redmond,113.23,2019-10-01 02:21:39+00:00,2019-10-01 05:28:39+00:00,187.00,средне


### 6.3 Обработка данных о времени покупки

In [ ]:
# Преобразуем время в числовой формат и очищаем от некорректных значений
purchase_journey_df['time_to_purchase_minutes'] = pd.to_numeric(
    purchase_journey_df['time_to_purchase_minutes'],
    errors='coerce'
)

# Фильтруем данные: оставляем только неотрицательное время
purchase_journey_df = purchase_journey_df[purchase_journey_df['time_to_purchase_minutes'] >= 0]
purchase_journey_df.head()

,user_id,product_id,category_code,brand,price,first_view_time,first_purchase_time,time_to_purchase_minutes,price_segment
0,553079594,15300165,None,transcend,6.12,2019-10-01 00:07:02+00:00,2019-10-01 05:44:39+00:00,337.62,дешево
1,547477164,19200010,construction.tools.saw,huter,84.66,2019-10-01 02:32:57+00:00,2019-10-01 05:57:07+00:00,204.17,дешево
2,546370010,15800018,None,karcher,540.02,2019-10-01 02:23:31+00:00,2019-10-01 05:45:27+00:00,201.93,дорого
3,551349791,1004903,electronics.smartphone,huawei,111.79,2019-10-01 02:38:51+00:00,2019-10-01 05:50:49+00:00,191.97,средне
4,514492564,12600087,appliances.kitchen.grill,redmond,113.23,2019-10-01 02:21:39+00:00,2019-10-01 05:28:39+00:00,187.00,средне


Отрицательные значения времени до покупки были исключены из анализа. Такие случаи могут возникать из-за отсутствующих событий и не поддаются корректной интерпретации

### 6.4 Статистика по ценовым сегментам

В таблице ниже представлены агрегированные данные времени до покупки
для каждого ценового сегмента, включая медиану, среднее значение
и межквартильный размах

In [ ]:
# Рассчитываем статистику времени до покупки для каждого ценового сегмента
time_stats_by_price_segment = (
    purchase_journey_df
    .groupby('price_segment')['time_to_purchase_minutes']
    .agg(
        count='count',
        mean='mean',
        median='median',
        q25=lambda x: x.quantile(0.25),
        q75=lambda x: x.quantile(0.75)
    )
    .reset_index()
    .round(1)
)

time_stats_by_price_segment

,price_segment,count,mean,median,q25,q75
0,дешево,942,9.3,2.8,1.2,6.7
1,дорого,404,6.2,1.6,0.7,3.6
2,очень дорого,175,3.8,1.4,0.7,3.0
3,средне,1709,8.1,2.4,1.1,5.6


## 7. Экспорт результатов
Сохраняем результаты анализа в CSV файлы для дальнейшей визуализации в Tableau

In [ ]:
# Сохраняем агрегированную статистику
time_stats_by_price_segment.to_csv('price_bucket_time.csv', index=False)
print("Файл 'price_bucket_time.csv' сохранен")

Файл 'price_bucket_time.csv' сохранен


In [ ]:
# Сохраняем детальные данные
purchase_journey_df[['price_segment', 'time_to_purchase_minutes']].to_csv(
    'time_to_purchase_raw.csv',
    index=False
)
print("Файл 'time_to_purchase_raw.csv' сохранен")

## 8. Основные выводы и рекомендации

- Покупка товара не всегда сопровождается добавлением в корзину
- Для товаров с низкой стоимостью характерно более длительное принятие решения, что может быть связано с просмотром и откладыванием покупки
- Дорогие товары чаще приобретаются быстрее, что говорит
  о более осознанном намерении
- Медианное время до покупки является более устойчивой метрикой,
  чем среднее значение, из-за асимметричного распределения данных

  **Автор:** Василева Алина - Pet Project для позиции Junior Product Analyst